# Zone Manager

> Coordinates keyboard navigation across multiple zones, modes, and actions.

In [ ]:
#| default_exp core.manager

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional

from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.modes import KeyboardMode, NAVIGATION_MODE
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.key_mapping import KeyMapping, ARROW_KEYS

## ZoneManager

The main coordinator that brings together zones, modes, actions, and key mappings.

In [ ]:
#| export
@dataclass
class ZoneManager:
    """Coordinates keyboard navigation across zones."""

    # Zones
    zones: tuple[FocusZone, ...]  # all focus zones

    # Zone switching
    prev_zone_key: str = "ArrowLeft"  # key to switch to previous zone
    next_zone_key: str = "ArrowRight"  # key to switch to next zone
    zone_switch_modifiers: frozenset[str] = field(
        default_factory=frozenset
    )  # modifiers for zone switching
    wrap_zones: bool = True  # wrap from last zone to first

    # Key mapping for navigation
    key_mapping: KeyMapping = field(
        default_factory=lambda: ARROW_KEYS
    )  # key-to-direction mapping

    # Initial state
    initial_zone_id: Optional[str] = None  # defaults to first zone

    # Modes
    modes: tuple[KeyboardMode, ...] = ()  # custom modes (navigation mode is implicit)
    default_mode: str = "navigation"  # mode to return to after exiting others

    # Actions
    actions: tuple[KeyAction, ...] = ()  # keyboard action bindings

    # Global callbacks (JS function names)
    on_zone_change: Optional[str] = None  # called when active zone changes
    on_mode_change: Optional[str] = None  # called when mode changes
    on_state_change: Optional[str] = None  # called on any state change (for persistence)

    # Input detection
    skip_when_input_focused: bool = True  # ignore keys in input/textarea
    input_selector: str = "input, textarea, select, [contenteditable]"  # elements to skip

    # HTMX integration
    htmx_settle_event: str = "htmx:afterSettle"  # event to reinitialize on

    # State exposure
    expose_state_globally: bool = False  # expose state on window object
    global_state_name: str = "keyboardNavState"  # name for global state
    state_hidden_inputs: bool = False  # write state to hidden inputs

    def __post_init__(self):
        """Validate configuration."""
        if not self.zones:
            raise ValueError("At least one zone is required")
        
        # Check for duplicate zone IDs
        zone_ids = [z.id for z in self.zones]
        if len(zone_ids) != len(set(zone_ids)):
            raise ValueError("Duplicate zone IDs detected")
        
        # Validate initial_zone_id
        if self.initial_zone_id and self.initial_zone_id not in zone_ids:
            raise ValueError(f"initial_zone_id '{self.initial_zone_id}' not found in zones")

    def get_zone(
        self,
        zone_id: str  # zone ID to find
    ) -> Optional[FocusZone]: # the zone or None
        """Get zone by ID."""
        for zone in self.zones:
            if zone.id == zone_id:
                return zone
        return None

    def get_initial_zone_id(self) -> str: # the initial zone ID
        """Get initial zone ID."""
        return self.initial_zone_id or self.zones[0].id

    def get_all_modes(self) -> tuple[KeyboardMode, ...]: # all modes including default
        """Get all modes including the default navigation mode."""
        return (NAVIGATION_MODE,) + self.modes

    def get_mode(
        self,
        mode_name: str  # mode name to find
    ) -> Optional[KeyboardMode]: # the mode or None
        """Get mode by name."""
        for mode in self.get_all_modes():
            if mode.name == mode_name:
                return mode
        return None

    def get_actions_for_context(
        self,
        zone_id: str,  # current zone
        mode_name: str  # current mode
    ) -> list[KeyAction]: # actions valid in this context
        """Get actions valid for given zone and mode."""
        return [
            action for action in self.actions
            if action.matches_context(zone_id, mode_name)
        ]

    def get_all_data_attributes(self) -> set[str]: # unique data attributes across all zones
        """Get all unique data attributes from all zones."""
        attrs = set()
        for zone in self.zones:
            attrs.update(zone.data_attributes)
        return attrs

    def to_js_config(self) -> dict: # JavaScript-compatible configuration
        """Convert to JavaScript configuration object."""
        return {
            "zones": [z.to_js_config() for z in self.zones],
            "zoneSwitching": {
                "prevKey": self.prev_zone_key,
                "nextKey": self.next_zone_key,
                "modifiers": list(self.zone_switch_modifiers),
                "wrap": self.wrap_zones,
            },
            "keyMapping": self.key_mapping.to_js_map(),
            "initialZoneId": self.get_initial_zone_id(),
            "modes": [m.to_js_config() for m in self.get_all_modes()],
            "defaultMode": self.default_mode,
            "actions": [a.to_js_config() for a in self.actions],
            "callbacks": {
                "onZoneChange": self.on_zone_change,
                "onModeChange": self.on_mode_change,
                "onStateChange": self.on_state_change,
            },
            "settings": {
                "skipWhenInputFocused": self.skip_when_input_focused,
                "inputSelector": self.input_selector,
                "htmxSettleEvent": self.htmx_settle_event,
                "exposeStateGlobally": self.expose_state_globally,
                "globalStateName": self.global_state_name,
                "stateHiddenInputs": self.state_hidden_inputs,
            },
        }

In [ ]:
# Test basic ZoneManager
from cjm_fasthtml_keyboard_navigation.core.navigation import LinearVertical

browser = FocusZone(
    id="browser",
    item_selector="tr.item",
    data_attributes=("job-id",)
)
queue = FocusZone(
    id="queue",
    item_selector="li.item"
)

manager = ZoneManager(
    zones=(browser, queue),
    actions=(
        KeyAction(key=" ", htmx_trigger="toggle"),
    )
)

assert manager.get_zone("browser") == browser
assert manager.get_zone("queue") == queue
assert manager.get_zone("invalid") is None
assert manager.get_initial_zone_id() == "browser"

In [ ]:
# Test modes
from cjm_fasthtml_keyboard_navigation.core.navigation import LinearHorizontal

split_mode = KeyboardMode(
    name="split",
    enter_key="Enter",
    navigation_override=LinearHorizontal()
)

manager_with_modes = ZoneManager(
    zones=(browser,),
    modes=(split_mode,)
)

all_modes = manager_with_modes.get_all_modes()
assert len(all_modes) == 2  # navigation + split
assert manager_with_modes.get_mode("navigation") is not None
assert manager_with_modes.get_mode("split") == split_mode

In [ ]:
# Test action filtering
actions = (
    KeyAction(key=" ", htmx_trigger="toggle"),  # all zones/modes
    KeyAction(key="Delete", htmx_trigger="delete", zone_ids=("queue",)),
    KeyAction(key="Enter", htmx_trigger="split", mode_names=("split",)),
)

manager = ZoneManager(zones=(browser, queue), actions=actions)

# Browser in navigation mode
browser_nav_actions = manager.get_actions_for_context("browser", "navigation")
assert len(browser_nav_actions) == 1  # only space toggle

# Queue in navigation mode
queue_nav_actions = manager.get_actions_for_context("queue", "navigation")
assert len(queue_nav_actions) == 2  # space + delete

# Any zone in split mode
split_actions = manager.get_actions_for_context("browser", "split")
assert len(split_actions) == 2  # space + enter

In [ ]:
# Test validation
import traceback

# Empty zones should fail
try:
    ZoneManager(zones=())
    assert False, "Should have raised ValueError"
except ValueError as e:
    assert "At least one zone" in str(e)

# Duplicate zone IDs should fail
try:
    ZoneManager(zones=(
        FocusZone(id="same"),
        FocusZone(id="same")
    ))
    assert False, "Should have raised ValueError"
except ValueError as e:
    assert "Duplicate" in str(e)

# Invalid initial zone should fail
try:
    ZoneManager(
        zones=(FocusZone(id="zone1"),),
        initial_zone_id="nonexistent"
    )
    assert False, "Should have raised ValueError"
except ValueError as e:
    assert "not found" in str(e)

In [ ]:
# Test JS config generation
config = manager.to_js_config()

assert len(config["zones"]) == 2
assert config["initialZoneId"] == "browser"
assert config["defaultMode"] == "navigation"
assert config["settings"]["skipWhenInputFocused"] == True

In [ ]:
# Test custom key mapping
from cjm_fasthtml_keyboard_navigation.core.key_mapping import WASD_KEYS

wasd_manager = ZoneManager(
    zones=(browser,),
    key_mapping=WASD_KEYS
)

config = wasd_manager.to_js_config()
assert config["keyMapping"]["w"] == "up"
assert config["keyMapping"]["s"] == "down"

In [ ]:
# Test data attributes collection
z1 = FocusZone(id="z1", data_attributes=("a", "b"))
z2 = FocusZone(id="z2", data_attributes=("b", "c"))

m = ZoneManager(zones=(z1, z2))
attrs = m.get_all_data_attributes()
assert attrs == {"a", "b", "c"}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()